In [1]:
import re
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df=pd.read_csv("anime.csv")
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [3]:
df.drop(["anime_id","rating","members"],axis=1,inplace=True)

In [4]:
name=[]
for i in df["name"]:
    name.append(i.lower())
df["name"]=name

In [5]:
df["content"]=df["name"]+" "+df["genre"]+" "+df["type"]+" "+df["episodes"]
df

,name,genre,type,episodes,content
0,kimi no na wa.,"Drama, Romance, School, Supernatural",Movie,1,"kimi no na wa. Drama, Romance, School, Superna..."
1,fullmetal alchemist: brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,"fullmetal alchemist: brotherhood Action, Adven..."
2,gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,"gintama° Action, Comedy, Historical, Parody, S..."
3,steins;gate,"Sci-Fi, Thriller",TV,24,"steins;gate Sci-Fi, Thriller TV 24"
4,gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,"gintama&#039; Action, Comedy, Historical, Paro..."
...,...,...,...,...,...
12289,toushindai my lover: minami tai mecha-minami,Hentai,OVA,1,toushindai my lover: minami tai mecha-minami H...
12290,under world,Hentai,OVA,1,under world Hentai OVA 1
12291,violence gekiga david no hoshi,Hentai,OVA,4,violence gekiga david no hoshi Hentai OVA 4
12292,violence gekiga shin david no hoshi: inma dens...,Hentai,OVA,1,violence gekiga shin david no hoshi: inma dens...


In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
content=df["content"]
content.info()

<class 'pandas.core.series.Series'>
RangeIndex: 12294 entries, 0 to 12293
Series name: content
Non-Null Count  Dtype 
--------------  ----- 
12210 non-null  object
dtypes: object(1)
memory usage: 96.2+ KB


In [8]:
data=[]
for i in content:
    lower=re.sub("[^a-zA-Z0-9 ]","",str(i))
    lower=lower.lower()
    data.append(lower)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vector=TfidfVectorizer()
values=vector.fit_transform(data)
values.toarray().shape

(12294, 12902)

In [10]:
vector.get_feature_names_out()

array(['00', '000', '0008', ..., 'zwei', 'zx', 'zz'], dtype=object)

In [11]:
from  sklearn.metrics.pairwise import cosine_similarity
cosine=cosine_similarity(values,values)

In [12]:
cosine

array([[1.        , 0.03378697, 0.        , ..., 0.02760503, 0.02266334,
        0.05978781],
       [0.03378697, 1.        , 0.07541995, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.07541995, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.02760503, 0.        , 0.        , ..., 1.        , 0.82098582,
        0.05706736],
       [0.02266334, 0.        , 0.        , ..., 0.82098582, 1.        ,
        0.04685149],
       [0.05978781, 0.        , 0.        , ..., 0.05706736, 0.04685149,
        1.        ]])

In [13]:
indices = pd.Series(df.index, index=df['name']).drop_duplicates()
def get_recommendations(title, cosine_sim=cosine):
    if title not in indices:
        return f"'{title}' not found in dataset."
        
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Top 5 excluding self
    anime_indices = [i[0] for i in sim_scores]
    
    return df['name'].iloc[anime_indices]

In [14]:
get_recommendations("kimi to boku. 2")

764      kimi to boku.
2299      kimi to boku
4747        to heart 2
4539          to heart
319     kimi ni todoke
Name: name, dtype: object

In [15]:
df.iloc[319]

name                                           kimi ni todoke
genre                  Romance, School, Shoujo, Slice of Life
type                                                       TV
episodes                                                   25
content     kimi ni todoke Romance, School, Shoujo, Slice ...
Name: 319, dtype: object

In [16]:
indices = pd.Series(df.index, index=df['name']).drop_duplicates()
print(indices)

name
kimi no na wa.                                            0
fullmetal alchemist: brotherhood                          1
gintama°                                                  2
steins;gate                                               3
gintama&#039;                                             4
                                                      ...  
toushindai my lover: minami tai mecha-minami          12289
under world                                           12290
violence gekiga david no hoshi                        12291
violence gekiga shin david no hoshi: inma densetsu    12292
yasuji no pornorama: yacchimae!!                      12293
Length: 12294, dtype: int64


In [26]:
values=pd.Series(df.index,index=df["name"])

In [28]:
values

name
kimi no na wa.                                            0
fullmetal alchemist: brotherhood                          1
gintama°                                                  2
steins;gate                                               3
gintama&#039;                                             4
                                                      ...  
toushindai my lover: minami tai mecha-minami          12289
under world                                           12290
violence gekiga david no hoshi                        12291
violence gekiga shin david no hoshi: inma densetsu    12292
yasuji no pornorama: yacchimae!!                      12293
Length: 12294, dtype: int64

In [ ]:
df.head()

In [ ]:
def remmondation(name,cosine_sim=coisne):
    if name not in values.index:
        return f"{name} not found in the animes try different one"
    
        